## <strong> <a style="color:black">LSTM Model Accuracy Comparison </a> </strong> 

---

|  |  |
|:---|:---|
|**Reading Time** |  10 minutes |
|**Packages used** | Defined with relevant DocStrings within repository   |
|**Model Hyperparameter** | Read trained model ./model/output/LSTM|
|**Outcome** | Comparison of the model accuracy on the real stock price history | 

---

There are two plots that I have plotted towards the end of the notebook to compare the model performance with the real world scenario. 

In the first plot, I have used trained model to predict next day return based on the updated new features by incorporating effect of new day data by each passing day in the features while in the second plot, I have plotted based on multi-step forecasting technique where we use the predicted variable as an input to predict in future, thus, all future values are predicted at time t=0.

---

In [1]:
import os
import pickle

import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import datetime as dt

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # just to remove clutter in the notebook

from pandas.tseries.offsets import BDay
from datetime import date, datetime, timedelta
from keras.layers.core import Dense, Dropout
from keras.layers import LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
os.chdir("..")

---

### <strong> <a style="color:black">Import self defined methods </a> </strong> 

All these methods are defined with appropriate DocStrings within the repository.

---

In [3]:
from model.utils.pre_processing_LSTM import create_features_and_target_split_lstm, standardize_and_limit_outliers_returns, get_features_for_multi_step_forecasting, add_technical_indicators
from model.core.LSTM import build_lstm_model_structure

### <strong> <a style="color:black">Get the data </a> </strong> 


In the following cells, I have defined the stock ticker, symbol and used yahoo finance to download the historical data. You can change the ticker, and symbol accordingly if wants to compare model performance for other stocks mentioned in static file in the directory ./model/static_data/

---

In [4]:
ticker = "SBIN.NS"
Stock = "SBI"
end_date = datetime.today()
start_date = date(2016, 1, 1)
df_hist = yf.download(ticker, start=start_date, end=end_date)
df_all_dates = pd.DataFrame(index=pd.bdate_range(start=start_date,end=end_date))
df_hist = pd.merge(df_all_dates, df_hist, how="left", left_index=True, right_index=True)
df_hist = df_hist.fillna(method='ffill')

[*********************100%***********************]  1 of 1 completed


### <strong> <a style="color:black">Static data used for Features and Train-test Split.</a> </strong> 

Define training data end date, validation period end date and number of past daily returns that are used as features to predict next day's return.

---

In [5]:
param = {'training_end': end_date - timedelta(seconds=2 * 365.2425 * 24 * 60 * 60),
             'validation_end': end_date - timedelta(seconds=1 * 365.2425 * 24 * 60 * 60),
             'past_day_returns_for_predicting': 21}

### <strong> <a style="color:black">Technical indicators as a features</a> </strong> 

Applied self defined method which used to create extra momenetum based technical indicators using tulipy package which will be used as features along with past daily returns to predict next day return.

---

In [7]:
df_hist = add_technical_indicators(df_hist)
df_hist['daily_returns'] = df_hist['Adj Close'].pct_change()
df_hist = df_hist.dropna()
dt_model = create_features_and_target_split_lstm(pd.DataFrame(df_hist['Adj Close']),
                                                param['past_day_returns_for_predicting'])
dt_model = pd.merge(dt_model, df_hist, how="left", left_index=True, right_index=True)
technical_indicator_features = ['RSI', 'MACD', 'UpperBollingerBand', 'LowerBollingerBand']

### <strong> <a style="color:black"> Data Pre-Processing and Hyper-Parameter setting</a> </strong> 

In the following cells, I have first alligned the training features and normalized it and stored the scaler to re-use it in the testing data set. Second step is to read the already trained model for output directory to use the tuned hyper parameters for fitting the model

---

In [10]:
X_train, Y_train, X_val, Y_val, scaler = standardize_and_limit_outliers_returns(dt_model, param['past_day_returns_for_predicting'], technical_indicator_features, **param)

In [11]:
X_train = np.asarray(X_train).astype(np.float32)
Y_train = np.asarray(Y_train).astype(np.float32)
new_x_tensor = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
new_y_tensor = Y_train.reshape(Y_train.shape[0], 1)

In [22]:
lstm_model = build_lstm_model_structure(num_features=new_x_tensor.shape[1], num_units=100)
lstm_model.fit(new_x_tensor, new_y_tensor, batch_size=100, epochs=300, verbose=True)

Epoch 1/300
14/14 [==============================] - 2s 49ms/step - loss: 72497.9141 - r_square: -28.8901
Epoch 2/300
14/14 [==============================] - 1s 49ms/step - loss: 72439.5000 - r_square: -28.8661
Epoch 3/300
14/14 [==============================] - 1s 69ms/step - loss: 72219.9609 - r_square: -28.7753
Epoch 4/300
14/14 [==============================] - 1s 73ms/step - loss: 67583.4766 - r_square: -26.8639
Epoch 5/300
14/14 [==============================] - 1s 63ms/step - loss: 59678.1758 - r_square: -23.6046
Epoch 6/300
14/14 [==============================] - 1s 52ms/step - loss: 54776.7539 - r_square: -21.5838
Epoch 7/300
14/14 [==============================] - 1s 56ms/step - loss: 47299.3359 - r_square: -18.5010
Epoch 8/300
14/14 [==============================] - 1s 58ms/step - loss: 31548.1309 - r_square: -12.0069
Epoch 9/300
14/14 [==============================] - 1s 66ms/step - loss: 13136.9395 - r_square: -4.4162
Epoch 10/300
14/14 [===========================

14/14 [==============================] - 1s 58ms/step - loss: 307.0305 - r_square: 0.8734
Epoch 81/300
14/14 [==============================] - 1s 56ms/step - loss: 380.5630 - r_square: 0.8431
Epoch 82/300
14/14 [==============================] - 1s 55ms/step - loss: 306.6627 - r_square: 0.8736
Epoch 83/300
14/14 [==============================] - 1s 56ms/step - loss: 310.7238 - r_square: 0.8719
Epoch 84/300
14/14 [==============================] - 1s 55ms/step - loss: 320.7417 - r_square: 0.8678
Epoch 85/300
14/14 [==============================] - 1s 57ms/step - loss: 296.6725 - r_square: 0.8777
Epoch 86/300
14/14 [==============================] - 1s 57ms/step - loss: 309.1882 - r_square: 0.8725
Epoch 87/300
14/14 [==============================] - 1s 56ms/step - loss: 333.6244 - r_square: 0.8625
Epoch 88/300
14/14 [==============================] - 1s 56ms/step - loss: 326.0797 - r_square: 0.8656
Epoch 89/300
14/14 [==============================] - 1s 57ms/step - loss: 294.5295 - 

14/14 [==============================] - 1s 56ms/step - loss: 181.9807 - r_square: 0.9250
Epoch 160/300
14/14 [==============================] - 1s 55ms/step - loss: 167.9967 - r_square: 0.9307
Epoch 161/300
14/14 [==============================] - 1s 56ms/step - loss: 213.3907 - r_square: 0.9120
Epoch 162/300
14/14 [==============================] - 1s 55ms/step - loss: 201.9723 - r_square: 0.9167
Epoch 163/300
14/14 [==============================] - 1s 55ms/step - loss: 235.7087 - r_square: 0.9028
Epoch 164/300
14/14 [==============================] - 1s 55ms/step - loss: 189.5335 - r_square: 0.9219
Epoch 165/300
14/14 [==============================] - 1s 55ms/step - loss: 202.6273 - r_square: 0.9165
Epoch 166/300
14/14 [==============================] - 1s 55ms/step - loss: 186.3069 - r_square: 0.9232
Epoch 167/300
14/14 [==============================] - 1s 55ms/step - loss: 178.7252 - r_square: 0.9263
Epoch 168/300
14/14 [==============================] - 1s 55ms/step - loss: 19

14/14 [==============================] - 1s 53ms/step - loss: 162.0396 - r_square: 0.9332
Epoch 238/300
14/14 [==============================] - 1s 53ms/step - loss: 159.5633 - r_square: 0.9342
Epoch 239/300
14/14 [==============================] - 1s 55ms/step - loss: 166.1106 - r_square: 0.9315
Epoch 240/300
14/14 [==============================] - 1s 54ms/step - loss: 152.6309 - r_square: 0.9371
Epoch 241/300
14/14 [==============================] - 1s 54ms/step - loss: 132.1219 - r_square: 0.9455
Epoch 242/300
14/14 [==============================] - 1s 53ms/step - loss: 141.2254 - r_square: 0.9418
Epoch 243/300
14/14 [==============================] - 1s 54ms/step - loss: 124.9945 - r_square: 0.9485
Epoch 244/300
14/14 [==============================] - 1s 54ms/step - loss: 183.1654 - r_square: 0.9245
Epoch 245/300
14/14 [==============================] - 1s 54ms/step - loss: 132.9896 - r_square: 0.9452
Epoch 246/300
14/14 [==============================] - 1s 54ms/step - loss: 15

In [23]:
lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 25, 100)           40800     
                                                                 
 dropout_6 (Dropout)         (None, 25, 100)           0         
                                                                 
 lstm_10 (LSTM)              (None, 25, 100)           80400     
                                                                 
 dropout_7 (Dropout)         (None, 25, 100)           0         
                                                                 
 lstm_11 (LSTM)              (None, 100)               80400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
 activation_3 (Activation)   (None, 1)                

In [24]:
dt_model['predicted_value'] = ""

In [25]:
df_training = dt_model[dt_model.index <= param['training_end']]
df_test = dt_model[dt_model.index > param['validation_end']]

---

### <strong> <a style="color:black"> Model Performance </a> </strong> 

**First Chart** : Plotting the prediction based on updating features on a daily basis. Issue in this approach is that we are using updated realized new features to predict next day return, and thus, we have to wait for the whole test period to end to compare the actual model performance

---

In [26]:
df_test_subset = df_test.copy()
df_test_subset['Actual'] = ""
df_test_subset['Predicted'] = ""
df_test_subset.iloc[0, df_test_subset.columns.get_loc('Actual')] = 100
df_test_subset.iloc[0, df_test_subset.columns.get_loc('Predicted')] = 100

In [27]:
rol_freq = param['past_day_returns_for_predicting']
for i in range(0, len(df_test_subset) - 1):
    features_ls = np.concatenate((df_test_subset.iloc[i, :rol_freq].values, df_test_subset.iloc[i][technical_indicator_features].values))
    features_ls = features_ls.reshape(-1,1)
    new_scaled_ls = scaler.transform(features_ls.T)
    new_scaled_ls = np.asarray(new_scaled_ls).astype(np.float32)
    new_scaled_ls = new_scaled_ls.reshape((new_scaled_ls.shape[0], new_scaled_ls.shape[1], 1))
    predicted_value = lstm_model.predict(new_scaled_ls)[0][0]
    # current features are predicting next day return
    df_test_subset.iloc[i+1, df_test_subset.columns.get_loc('predicted_value')] = predicted_value
#     df_test_subset.iloc[i+1, df_test_subset.columns.get_loc('Predicted')] = (1 + predicted_return) * df_test_subset.iloc[i, df_test_subset.columns.get_loc('Predicted')]
    # actual trend
#     actual_return = df_test_subset.iloc[i+1, df_test_subset.columns.get_loc('actual_returns')]
#     df_test_subset.iloc[i+1, df_test_subset.columns.get_loc('Actual')] = (1 + actual_return) * df_test_subset.iloc[i, df_test_subset.columns.get_loc('Actual')]

1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step


In [ ]:
fig, ax = plt.subplots(figsize = (18,10))
ax.plot(df_test_subset.index, df_test_subset['Actual'], label="Actual")
ax.plot(df_test_subset.index, df_test_subset['Predicted'], label="Predicted")

ax.set_title(f"{Stock} Trading vs Prediction (Updating features to predict next day return on a daily basis)", fontsize=20, color="black")
ax.set_xlabel("Date", fontsize=25, color="black")
ax.set_ylabel("Rupees", fontsize=25, color="black")
ax.tick_params(axis='both', labelsize=20)

ax.set_frame_on(True)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

plt.grid(b=True, color="aqua", alpha=0.5, linestyle='-.')
plt.legend(loc='upper right', prop={'size': 25})
# plt.style.use('classic')
plt.show()


---

### <strong> <a style="color:black"> 6. Multi-Step Forecasting </a> </strong> 

**Second Chart:** It uses the predicted variables as an input features for the next day return and,thus, making all future prediction at time t=0 

---

In [ ]:
df_test_multi_step = dt_model[dt_model.index > param['validation_end']]
df_test_multi_step['Predicted Adj Close'] = df_test_multi_step['Adj Close'] # will update it while looping
end_date = dt.datetime.today()
period_start_date = end_date - dt.timedelta(days=21)

In [ ]:
rol_freq = param['past_day_returns_for_predicting']
predicted_dates = pd.bdate_range(start=period_start_date,end=end_date)

In [ ]:
for date in predicted_dates[:-1]:
    price_trend_before_date = pd.DataFrame(df_test_multi_step.loc[df_test_multi_step.index < date, 'Predicted Adj Close'])
    df = price_trend_before_date.copy()
    features_ls = get_features_for_multi_step_forecasting(df, rol_freq, technical_indicator_features)
    features_ls = features_ls.reshape(-1,1)
    new_scaled_ls = scaler.transform(features_ls.T)
    new_scaled_ls = np.asarray(new_scaled_ls).astype(np.float32)
    new_scaled_ls = new_scaled_ls.reshape((new_scaled_ls.shape[0], new_scaled_ls.shape[1], 1))
    predicted_return = lstm_model.predict(new_scaled_ls)[0][0]
    next_working_date = (date + BDay(1)).replace(hour=0, minute=0, second=0, microsecond=0)
    date = date.replace(hour=0, minute=0, second=0, microsecond=0)
    df_test_multi_step.loc[next_working_date, 'predicted_returns'] = predicted_return
    df_test_multi_step.loc[next_working_date, 'Predicted Adj Close'] = (1 + predicted_return) * df_test_multi_step.loc[date, 'Predicted Adj Close']

In [ ]:
fig, ax = plt.subplots(figsize = (18,10))
ax.plot(df_test_multi_step.index, df_test_multi_step['Adj Close'], label="Actual")
ax.plot(df_test_multi_step.index, df_test_multi_step['Predicted Adj Close'], label="Predicted")

ax.set_title(f"{Stock} Stock trend Comparison (Multi-Step forecasting at time t=0)", fontsize=20, color="black")
ax.set_xlabel("Date", fontsize=25, color="black")
ax.set_ylabel("Rupees", fontsize=25, color="black")
ax.tick_params(axis='both', labelsize=20)

ax.set_frame_on(True)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

plt.grid(b=True, color="aqua", alpha=0.5, linestyle='-.')
plt.legend(loc='upper right', prop={'size': 25})
plt.axvline(x=predicted_dates[0], color="r", linestyle="--")
plt.show()

In [21]:
df_test_subset.to_clipboard()

In [ ]:
y_pred = lstm_model.predict(new_x_tensor)

In [ ]:
r2_score(y_pred, new_y_tensor)

In [28]:
df_test_subset.to_clipboard()